## Mexican Hat Wavelet Analysis For Topographic Roughness
* The original MATLAB code was developed from Dr. Adam M. Booth (Portland State Univeristy).  
    * Citations:  
        * Booth, A.M., Roering, J.J., Perron, J.T., 2009. Automated landslide mapping using spectral analysis and high-resolution topographic data: Puget Sound lowlands, Washington, and Portland    Hills, Oregon. Geomorphology 109, 132-147. https://doi.org/10.1016/j.geomorph.2009.02.027  
        * Booth, A.M., LaHusen, S.R., Duvall, A.R., Montgomery, D.R., 2017. Holocene history of deep-seated landsliding in the North Fork Stillaguamish River valley from surface roughness analysis, radiocarbon dating, and numerical landscape evolution modeling. Journal of Geophysical Research: Earth Surface 122, 456-472. https://doi.org/10.1002/2016JF003934  
* This MATLAB code was later adapted and revised by Dr. Sean R. LaHusen & Erich N. Herzig (Univeristy of Washington)
    * Citations:  
       * LaHusen, S.R., Duvall, A.R., Booth, A.M., Montgomery, D.R., 2016. Surface roughness dating of long-runout landslides near Oso, Washington (USA), reveals persistent postglacial hillslope instability. Geology 44, 111-114. https://doi.org/10.1130/G37267.1  
       * LaHusen, S.R., Duvall, A.R., Booth, A.M., Grant, A., Mishkin, B.A., Montgomery, D.R., Struble, W., Roering, J.J., Wartman, J., 2020. Rainfall triggers more deep-seated landslides than Cascadia earthquakes in the Oregon Coast Range, USA. Science Advances 6, eaba6790. https://doi.org/10.1126/sciadv.aba6790  
       * Herzig et al. (2023 in print) Bulletin of the Seismological Society of America. (details TBA)  
* In November, 2023; this is code translated and optimized into this python version by Dr. Larry Syu-Heng Lai (Univeristy of Washington)  
    * Citations: TBA  

### Initial setup

In [1]:
import numpy as np
import scipy.signal
import rasterio
from rasterio.enums import Resampling

### Read a DEM

In [2]:
# Define paths
input_tif_path = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/cropped swWA DEM.tif'
output_tif_path = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/cropped_swWA_mexhat_py.tif'

# Read the input GeoTIFF
with rasterio.open(input_tif_path) as src:
    dem = src.read(1)  # Read the first band into a 2D array
    meta = src.meta

See coordinate system info of the GeoTIFF

In [3]:
# Open the GeoTIFF file
with rasterio.open(input_tif_path) as src:
    # Read the CRS
    crs = src.crs
    
    # Print the CRS information
    print(f"CRS: {crs}")
    print(f"CRS as WKT: {crs.wkt}")
    print(f"CRS as PROJ string: {crs.to_proj4()}")
    print(f"CRS as EPSG code: {crs.to_epsg()}")
    print(f"CRS as dictionary: {crs.to_dict()}")

CRS: EPSG:32149
CRS as WKT: PROJCS["NAD83 / Washington South",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",45.3333333333333],PARAMETER["central_meridian",-120.5],PARAMETER["standard_parallel_1",47.3333333333333],PARAMETER["standard_parallel_2",45.8333333333333],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32149"]]
CRS as PROJ string: +init=epsg:32149
CRS as EPSG code: 32149
CRS as dictionary: {'init': 'epsg:32149'}


### Mexican Hat Wavelet Analysis Function (original slower ver)

In [4]:
def conv2_mexh(dem, a, dx):
    """
    Perform the 2D Continuous Wavelet Transform using the Mexican Hat wavelet.
    
    :param dem: Digital elevation model (2D numpy array).
    :param a: Wavelet scale.
    :param dx: Grid spacing.
    :return: Tuple of (C, frq, wave), the wavelet coefficients and frequencies.
    """
    # Generate the Mexican Hat wavelet kernel at wavelet scale a
    sz = int(np.ceil(8 * a))  # Kernel size
    X, Y = np.meshgrid(np.arange(-sz, sz+1), np.arange(-sz, sz+1))

    # Scaled Mexican Hat wavelet (psi)
    psi = (-1 / (np.pi * (a * dx)**4)) * (1 - (X**2 + Y**2) / (2 * a**2)) * np.exp(-(X**2 + Y**2) / (2 * a**2))

    # Convolve dem with psi
    C = dx**2 * scipy.signal.convolve2d(dem * 0.3048, psi, mode='same')

    # Mask edge effects with NaN values
    fringeval = int(np.ceil(a * 4))
    C[:fringeval, :] = np.nan
    C[-fringeval:, :] = np.nan
    C[:, :fringeval] = np.nan
    C[:, -fringeval:] = np.nan

    # Frequency and wavelength calculations
    wave = 2 * np.pi * dx * a / np.sqrt(5 / 2)  # Wavelength
    frq = 1 / wave  # Frequency
    
    return C, frq, wave

Set parameters

In [5]:
a = 4.1  # Wavelet scale
dx = 1.8288  # Grid spacing

Perform the function

In [ ]:
C, frq, wave = conv2_mexh(dem, a, dx)

Output data into GeoTIFF

In [ ]:
# Define georeference system
coord_ref_sys_code = 32149 #NAD38 Washington South
# coord_ref_sys_code = 32610 #WGS84_UTM Zone 10N

# Prepare the metadata for writing the output GeoTIFF
meta.update({
    'dtype': 'float32',
    'nodata': np.nan,
    'crs': f'EPSG:{coord_ref_sys_code}'
})

# Write the result to a new GeoTIFF
with rasterio.open(output_tif_path, 'w', **meta) as dst:
    dst.write(C.astype(np.float32), 1)  # Write the computed C as the first band

### Mexican Hat Wavelet Analysis Function - Optimized using Fast Fourier Transform (FFT)

In [4]:
# Use FFT convolution for better performance on large arrays
def conv2_mexh_fft(dem, a, dx):
    """
    Perform the 2D Continuous Wavelet Transform using the Mexican Hat wavelet.
    
    :param dem: Digital elevation model (2D numpy array).
    :param a: Wavelet scale.
    :param dx: Grid spacing.
    :return: Tuple of (C, frq, wave), the wavelet coefficients and frequencies.
    """
    # Kernel size, assuming the wavelet decays to 0 at the edges
    sz = int(np.ceil(8 * a))  
    X, Y = np.meshgrid(np.arange(-sz, sz+1), np.arange(-sz, sz+1))

    # Scaled Mexican Hat wavelet (psi)
    psi = (-1 / (np.pi * (a * dx)**4)) * (1 - (X**2 + Y**2) / (2 * a**2)) * np.exp(-(X**2 + Y**2) / (2 * a**2))

    # Convolve dem with psi using FFT for speed optimization
    C = scipy.signal.fftconvolve(dem, psi, mode='same')

    # Frequency and wavelength calculations
    wave = 2 * np.pi * dx * a / np.sqrt(5 / 2)  # Wavelength
    frq = 1 / wave  # Frequency
    
    return C, frq, wave

Set parameters

In [5]:
a = 4.1  # Wavelet scale
dx = 1.8288  # Grid spacing

##### Large DEM to be processed in chunks

In [8]:
# Define a suitable chunk size depending on your system's memory
chunk_size = 20000  # Example size, adjust based on your system's capability

# Initialize an empty array to store the wavelet coefficients
C_full = np.empty(dem.shape, dtype=np.float32)

# Perform the wavelet transform in chunks to avoid memory issues
for i in range(0, dem.shape[0], chunk_size):
    for j in range(0, dem.shape[1], chunk_size):
        # Extract the chunk of the DEM to process
        dem_chunk = dem[i:i+chunk_size, j:j+chunk_size]

        # Perform the wavelet transform on the chunk
        C_chunk, frq, wave = conv2_mexh_fft(dem_chunk, a, dx)

        # Store the result in the appropriate part of the full array
        C_full[i:i+chunk_size, j:j+chunk_size] = C_chunk

##### Perform the wavelet transform on the entire dataset (for smaller DEM file)

In [7]:
C_full, frq, wave = conv2_mexh_fft(dem, a, dx)

: 

Output data into GeoTIFF (Optimzation made for faster writing)    
* Enabling geotiff compression to reduce writing time
* Allowing Tile-based writing if needed (disabled for now)
* Enabling parallel writing - writing data into chunks/blocks  
* Enabling BIGTIFF parameter to allow writing a large GeoTIFF

In [9]:
# Define georeference system
coord_ref_sys_code = 32149 #NAD38 Washington South
#coord_ref_sys_code = 32610 #WGS84_UTM Zone 10N

# Prepare the metadata for writing the output GeoTIFF
meta.update({
    'dtype': 'float32',
    'nodata': np.nan,
    'crs': f'EPSG:{coord_ref_sys_code}',
    'compress': 'lzw',  # Using LZW compression
    'tiled': True,      # Writing in tiles
    'blockxsize': 256,  # Block size (adjust as needed)
    'blockysize': 256,
    'BIGTIFF': 'YES'    # Explicitly use BigTIFF format
})

# Write the result to a new GeoTIFF
with rasterio.open(output_tif_path, 'w', **meta) as dst:
    dst.write(C_full, 1)  # Write the computed C_full as the first band